<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/homework/DZ_Light_%D0%9E%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B0_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%BD%D0%BD%D0%BE%D0%B9_%D1%80%D0%B5%D1%87%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашнее задание**

Протестируйте пять любых зашумленных аудио файлов. Используйте любые модели, на свое усмотрение.
Используйте количественную характеристику качества распознавания речи - WER.

Напишите выводы.

In [ ]:
# Ваше решение
#установка необходимых библиотек
!pip install pydub
!pip install noisereduce
!pip install git+https://github.com/openai/whisper.git
!pip3 install vosk
!pip install pyspellchecker
!pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hw74w1nw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hw74w1nw
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#Основные импорты и загрузки
import noisereduce as nr
from spellchecker import SpellChecker
import soundfile as sf
import os
import whisper
import vosk
import wave
from pydub import AudioSegment
import gdown
import re
import json
import string

Подготовка данных + noisereduce

In [ ]:
# Подгружаем гугл-диск. Там лежат сохраненные файлы

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Прослушаем оригиналы

In [ ]:
# Послушаем экземпляр audio_2023-11-20_18-06-10.ogg в оригинальном качестве
audio_file = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset/audio_2023-11-22_18-37-32.ogg"
sample = AudioSegment.from_file(file=audio_file, format='ogg')
sample

In [ ]:
# Послушаем экземпляр audio_2023-11-20_18-06-10.ogg в оригинальном качестве
audio_file = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset/audio_2023-11-20_18-10-51.ogg"
sample = AudioSegment.from_file(file=audio_file, format='ogg')
sample

In [ ]:
# Послушаем экземпляр audio_2023-11-20_18-06-10.ogg в оригинальном качестве
audio_file = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset/audio_2023-12-01_14-26-07.ogg"
sample = AudioSegment.from_file(file=audio_file, format='ogg')
sample

In [ ]:
# Послушаем экземпляр audio_2023-11-20_18-06-10.ogg в оригинальном качестве
audio_file = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset/audio_2023-12-01_09-15-47.ogg"
sample = AudioSegment.from_file(file=audio_file, format='ogg')
sample

In [ ]:
# Послушаем экземпляр audio_2023-11-20_18-06-10.ogg в оригинальном качестве
audio_file = "/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset/audio_2023-12-01_09-16-16.ogg"
sample = AudioSegment.from_file(file=audio_file, format='ogg')
sample

В данной работе проверим, какую ценность представляет данный инструмент для задачи распознавания речи.

Модель vosk, которую мы будем тестировать, не обрабатывает файлы .ogg, поэтому переводим в формат .wav. Процесс занимает более 4 минут, и итоговая папка весит 1.6 гб

In [ ]:
new_folder = '/content/Датасет_аудио_wav'

In [ ]:
os.makedirs(new_folder, exist_ok=True)

org_folder = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Voice_filters/full_dataset'

for file_name in os.listdir(org_folder):
  if file_name.endswith(".ogg"):
    file_path = os.path.join(org_folder, file_name)
    audio_file = AudioSegment.from_ogg(file_path)
    new_file_name = os.path.splitext(file_name)[0] + ".wav"
    new_file_path = os.path.join(new_folder, new_file_name)
    audio_file.export(new_file_path, format="wav")

In [ ]:
# Путь к датасету с исходными аудио в wav формате
dataset_path = new_folder

Функции

In [ ]:
# Функция транскрибации с помощью vosk
def vosk_transcribation(filepath, model, rate):
    '''
    file_path - путь к аудио файлу с расширением .wav
    model - используемая модель vosk
    rate - частота оцифровки
    '''
    # считываем аудио файл
    with open(filepath, "rb") as f:
        data = f.read()

    # создаем экземпляр класса vosk.KaldiRecognizer и передаем ему модель и частоту оцифровки
    recognizer = vosk.KaldiRecognizer(model, rate)

    # распознаем речь, передавая буфер аудиофайла в метод AcceptWaveform объекта KaldiRecognizer
    recognizer.AcceptWaveform(data)

    # получаем результат
    recognized_data = json.loads(recognizer.Result())

    return recognized_data["text"]

In [ ]:
# Функция исправления ошибок в словах
def correct_spelling(text):

    # Создаем объект SpellChecker
    spell = SpellChecker(language='ru')

    # Разделяем строку на слова и знаки препинания
    words = re.findall(r'\w+|[^\w\s]', text)

    # Записываем все слова и знаки препинания в список потенциально ошибочных, будем их проверять и при необходимости исправлять
    misspelled = spell.unknown(words)

    result = []

    # Составляем новый список с исправленными словами, либо с исходными, если для них не нашлось варианта исправления
    for word in words:
        if word in misspelled:

            # Получаем самое вероятное исправление, если это возможно
            corrected = spell.correction(word)


            if corrected:
                # Если вариант исправления найден, то добавляем его в список
                result.append(corrected)
            else:
                # Если нет - то исходное слова
                result.append(word)
        else:
            result.append(word)

    # Объединяем список в строку
    result = ' '.join(result)

    # Убираем пробелы перед знаками препинания
    return re.sub(r'\s+([^\w\s])', r'\1', result)

In [ ]:
# Функции выполнения транскрибации оригинальных и очищенных от шума файлов,
# с применением и без исправления ошибок в словах

# Папка для очищенных от шума аудиофайлов
reduced_noise_dir = 'Очищенные'
os.makedirs(reduced_noise_dir, exist_ok=True)


def run_whisper(model_whisper, n):
    '''
    model_whisper - модель whisper
    n - количество обрабатываемых аудио файлов
    '''
    # проходимся по файлам датасета
    for i, file_name in enumerate(sorted(os.listdir(dataset_path))):

          # условие прекращения цикла по количеству файлов
          if i == n:
              break

          # обращаемся к исходному аудио файлу
          orig_audio_file = os.path.join(dataset_path, file_name)
          print(file_name)

          # убираем шум с помощью библиотеки reduce_noise
          data, rate = sf.read(orig_audio_file)
          reduced_noise = nr.reduce_noise(y=data, sr=rate)
          reduced_noise_file = os.path.join(reduced_noise_dir, file_name)

          # сохраняем очищенный файл в соответствующую папку
          sf.write(reduced_noise_file, reduced_noise, rate)

          # WHISPER. ИСХОДНОЕ АУДИО
          result = model_whisper.transcribe(orig_audio_file)
          result_text = result["text"].strip()
          print('Wisper. Оригинал:'.ljust(30), result_text)

          # выполнение коррекция правописания
          corrected_text = correct_spelling(result_text)
          print('Wisper. Оригинал (корр): '.ljust(30), corrected_text)


          # WHISPER. ОЧИЩЕННОЕ АУДИО
          result = model_whisper.transcribe(reduced_noise_file)
          result_text = result["text"].strip()
          print('Wisper. Очищенный:'.ljust(30), result_text)

          # выполнение коррекция правописания
          corrected_text = correct_spelling(result_text)
          print('Wisper. Очищенный (корр):  '.ljust(30), corrected_text)


          print()

def run_vosk(model_vosk, n):
    '''
    model_vosk - модель vosk
    n - количество обрабатываемых аудио файлов
    '''
    # проходимся по файлам датасета
    for i, file_name in enumerate(sorted(os.listdir(dataset_path))):

          # условие прекращения цикла по количеству файлов
          if i == n:
              break

          # обращаемся к исходному аудио файлу
          orig_audio_file = os.path.join(dataset_path, file_name)
          print(file_name)

          # убираем шум с помощью библиотеки reduce_noise
          data, rate = sf.read(orig_audio_file)
          reduced_noise = nr.reduce_noise(y=data, sr=rate)
          reduced_noise_file = os.path.join(reduced_noise_dir, file_name)

          # сохраняем очищенный файл в соответствующую папку
          sf.write(reduced_noise_file, reduced_noise, rate)

          # VOSK. ИСХОДНОЕ АУДИО
          result_text = vosk_transcribation(orig_audio_file, model_vosk, rate)
          print('Vosk. Оригинал:'.ljust(30), result_text)

          # выполнение коррекция правописания
          corrected_text = correct_spelling(result_text)
          print('Vosk. Оригинал (корр): '.ljust(30), corrected_text)


          # VOSK. ОЧИЩЕННОЕ АУДИО
          result_text = vosk_transcribation(reduced_noise_file, model_vosk, rate)
          print('Vosk. Очищенный:'.ljust(30), result_text)

          # выполнение коррекция правописания
          corrected_text = correct_spelling(result_text)
          print('Vosk. Очищенный (корр): '.ljust(30), corrected_text)

          print()

Эксперименты

whisper tiny (72.1 Мб)

In [ ]:
# Загружаем модель whisper tiny (72.1 Мб)
model_whisper = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 72.9MiB/s]


In [ ]:
# Количество экземпляров = 20
run_whisper(model_whisper, 20)

audio_2023-11-20_18-04-29.wav
Wisper. Оригинал:              Моа, закажи нам, пожалуйста, вилеты, Москву и Краснодара. Ятого дикабря возвращаемся 12 дикабря. Спасибо!
Wisper. Оригинал (корр):       Моа, закажи нам, пожалуйста, билеты, Москву и Краснодара. Ятого декабря возвращаемся 12 декабря. Спасибо!
Wisper. Очищенный:             У нас закажено пожалост вилеты, мастюгу из краснадара. Я этого дикообряю, возвращаемся 12 дикообряю. Спасибо.
Wisper. Очищенный (корр):      У нас закажем пожалост билеты, мастюгу из краснадара. Я этого дикообряю, возвращаемся 12 дикообряю. Спасибо.

audio_2023-11-20_18-04-59.wav
Wisper. Оригинал:              Какие есть варианты добраться? 15 декабря из краснодарова на Восибирск. Обратный возраст, 19 декабря посвожешь один.
Wisper. Оригинал (корр):       Какие есть варианты добраться? 15 декабря из краснодарова на Восибирск. Обратный возраст, 19 декабря поможешь один.
Wisper. Очищенный:             Какие есть варианты? Табрадс 15 декабря из краснотара по р

Транскрибация так себе. Многое не распознано.
Попробуем Wisper medium. Качество распознавания будет намного лучше.

In [ ]:
# Загружаем модель whisper medium (1.42 Гб)
model_whisper = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:47<00:00, 32.1MiB/s]


In [ ]:
# Количество экземпляров = 20
run_whisper(model_whisper, 20)

audio_2023-11-20_18-04-29.wav
Wisper. Оригинал:              Захажи нам, пожалуйста, билеты в Москву из Краснодара 5 декабря, возвращаемся 12 декабря. Спасибо!
Wisper. Оригинал (корр):       Захажи нам, пожалуйста, билеты я Москву из Краснодара 5 декабря, возвращаемся 12 декабря. Спасибо!
Wisper. Очищенный:             Захажи нам, пожалуйста, билеты в Москву из Краснодара 5 декабря. Возвращаемся 12 декабря. Спасибо.
Wisper. Очищенный (корр):      Захажи нам, пожалуйста, билеты я Москву из Краснодара 5 декабря. Возвращаемся 12 декабря. Спасибо.

audio_2023-11-20_18-04-59.wav
Wisper. Оригинал:              Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск обратно обратный возврат 19 декабря пассажир 1
Wisper. Оригинал (корр):       Какие есть варианты добраться 15 декабря из Краснодара я Новосибирск обратно обратный возврат 19 декабря пассажир 1
Wisper. Очищенный:             Какие есть варианты добраться 15 декабря из Краснодарова, Новосибирск обратно обратный возврат

Еще лучше отработает Wisper large.  По опыту стажировок знаю , что будет почти идеальный текст.

In [ ]:
# Загружаем модель whisper large (1.42 Гб)
model_whisper = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:57<00:00, 53.5MiB/s]


In [ ]:
# Количество экземпляров = 20
run_whisper(model_whisper, 20)

audio_2023-11-20_18-04-29.wav
Wisper. Оригинал:              Захожи нам пожалуйста билеты в Москву из Краснодара 5 декабря. Возвращаемся 12 декабря. Спасибо.
Wisper. Оригинал (корр):       Захожи нам пожалуйста билеты я Москву из Краснодара 5 декабря. Возвращаемся 12 декабря. Спасибо.
Wisper. Очищенный:             Захожи нам пожалуйста в билеты в Москву из Краснодара 5 декабря, возвращаемся 12 декабря. Спасибо!
Wisper. Очищенный (корр):      Захожи нам пожалуйста я билеты я Москву из Краснодара 5 декабря, возвращаемся 12 декабря. Спасибо!

audio_2023-11-20_18-04-59.wav
Wisper. Оригинал:              какие есть варианты добраться 15 декабря из краснодара в новосибирск обратно обратный возврат 19 декабря пассажир 1
Wisper. Оригинал (корр):       какие есть варианты добраться 15 декабря из краснодара я новосибирск обратно обратный возврат 19 декабря пассажир 1
Wisper. Очищенный:             Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря.

vosk-model-small-ru-0.22 (46 Мб)

In [ ]:
# Скачиваем модель vosk
gdown.download('https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip')
!unzip -q vosk-model-ru-0.22.zip

Downloading...
From: https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
To: /content/vosk-model-ru-0.22.zip
100%|██████████| 1.56G/1.56G [00:16<00:00, 97.2MB/s]


replace vosk-model-ru-0.22/graph/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Чтобы избежать переполнения ОЗУ, необходимо перезагрузить среду!!! CTRL+M

или Среда выполнения > правой кнопкой мыши > перезапустить сеанс > перезагрузить

In [ ]:
# Загружаем модель vosk на русском языке(1.56 Гб)
model_vosk = vosk.Model("vosk-model-ru-0.22")

In [ ]:
# Количество экземпляров = 20
run_vosk(model_vosk, 20)

audio_2023-11-20_18-04-29.wav
Vosk. Оригинал:                
Vosk. Оригинал (корр):         
Vosk. Очищенный:               закажи нам пожалуйста билеты в москву из краснодара пятого декабря возвращаемся двенадцатого декабря
Vosk. Очищенный (корр):        закажи нам пожалуйста билеты я москва из краснодара пятого декабря возвращаемся двенадцатого декабря

audio_2023-11-20_18-04-59.wav
Vosk. Оригинал:                
Vosk. Оригинал (корр):         
Vosk. Очищенный:               какие есть варианты добраться пятнадцатого декабря из краснодара по новосибирск обратно обратный возврат девятнадцатого декабря
Vosk. Очищенный (корр):        какие есть варианты добраться пятнадцатого декабря из краснодара по новосибирск обратно обратный возврат девятнадцатого декабря

audio_2023-11-20_18-05-06.wav
Vosk. Оригинал:                
Vosk. Оригинал (корр):         
Vosk. Очищенный:               подай-ка пожалуйста результаты маршрута в январе с пятнадцатого по двадцатое до петербурга и обратно па

На мой взгляд лучшие результаты транскрибации показала модель Wisper large.

импортируем wer

In [ ]:
from jiwer import wer

Вычисляем WER для 5 файлов по транскрибации Wisper large

№ 1

In [ ]:
# имеется строка Whisper large
text = 'маршрут проложи пожалуйста на 20 января с москвы до ярославль'

In [ ]:
# и строка с правильным написанием
text_correct = 'маршрут проложи пожалуйста на 20 января с москвы до ярославля'

In [ ]:
# Выполняем оценку
wer_metric = wer(text_correct, text)
wer_metric

0.1

№ 2

In [ ]:
# имеется строка Whisper large
text = 'Так, нам с моим другом нужна поездка из Касандара в Ростов за 3000 рублей. На двоих, то есть.'

In [ ]:
# и строка с правильным написанием
text_correct = 'Так, нам с моим другом нужна поездка из Касандара в Ростов за 3000 рублей. На двоих, то есть.'

In [ ]:
# Выполняем оценку
wer_metric = wer(text_correct, text)
wer_metric

0.0

№ 3


In [ ]:
# имеется строка Whisper large
text = 'Короче, завтра с кем-то хотим в Москву из Перми. Вот. 10 тысяч рублей. 10-15, наверное. Саня, у тебя есть пятак? Саня, у тебя есть пятак?'

In [ ]:
# и строка с правильным написанием
text_correct = 'Короче, завтра с кентом хотим в Москву из Перми. Вот. 10 тысяч рублей. 10-15, наверное. Саня, у тебя есть пятак? Саня, у тебя есть пятак?'

In [ ]:
# Выполняем оценку
wer_metric = wer(text_correct, text)
wer_metric

0.04

№ 4

In [ ]:
# имеется строка Whisper large очищенная,
# получается очищение на модель влияет отрицательно
text = 'Я информирую на 25 число билет из Перми до Москвы за 5000 рублей. Угу, да.'

In [ ]:
# и строка с правильным написанием кстати оригинал из Wisper large не нужно даже править.
text_correct = 'Я планирую на 25 число билет из Перми до Москвы за 5000 рублей. Угу, да.'

In [ ]:
# Выполняем оценку
wer_metric = wer(text_correct, text)
wer_metric

0.06666666666666667

№ 5

In [ ]:
# имеется строка Whisper large
text = 'Так, нам я моим другом нужна поездка из Касандара я Ростов за 3000 рублей. На двоих, то есть.'

In [ ]:
# и строка с правильным написанием
text_correct = 'Так, нам я моим другом нужна поездка из Касандара в Ростов за 3000 рублей. На двоих, то есть.'

In [ ]:
# Выполняем оценку
wer_metric = wer(text_correct, text)
wer_metric

0.05555555555555555

Приемлемым считается значение WER не более 0.15. Чем меньше WER, тем лучше. WER, подсчитанный для 2 одинаковых фраз, выдаст 0

In [ ]:
# Выполняем оценку двух одинаковых строк
wer_metric = wer(text_correct, text_correct)
wer_metric

0.0

Мои выводы :            

Самыми оптимальными по качеству транскрибации текста оказались модели Whisper large и Whisper с промтом, Whisper medium.

Это также подтвердила стажировка.